In [32]:
import torch_geometric
import torch
import torch.nn as nn
import torch_geometric.nn as gnn # note the difference from nn!
from torch_geometric.nn import GCNConv, global_sort_pool # NOTE: global_sort_pool!!!
from torch_geometric.utils import degree
import torch.nn.functional as F
from torch.nn import Conv1d, MaxPool1d, Linear, Dropout
from torch_geometric.utils import remove_self_loops


from torch_geometric.datasets import TUDataset
import pandas as pd
import numpy as np

In [24]:
class Indegree(object):
    r"""Adds the globally normalized node degree to the node features.
    Args:
        cat (bool, optional): If set to :obj:`False`, all existing node
            features will be replaced. (default: :obj:`True`)
    """

    def __init__(self, norm=True, max_value=None, cat=True):
        self.norm = norm
        self.max = max_value
        self.cat = cat

    def __call__(self, data):
        col, x = data.edge_index[1], data.x
        deg = degree(col, data.num_nodes)

        if self.norm:
            deg = deg / (deg.max() if self.max is None else self.max)

        deg = deg.view(-1, 1)

        if x is not None and self.cat:
            x = x.view(-1, 1) if x.dim() == 1 else x
            data.x = torch.cat([x, deg.to(x.dtype)], dim=-1)
        else:
            data.x = deg

        return data

    def __repr__(self):
        return '{}(norm={}, max_value={})'.format(self.__class__.__name__, self.norm, self.max)

In [25]:
prot = TUDataset('.', 'PROTEINS', pre_transform=Indegree(), use_node_attr=True, ) # this downloads the PROTEINS data set to the current directory and 
# and loads it to the variable prot, prot is now a torch_geometric.Dataset object

Processing...
Done!


In [27]:
prot[0]

Data(edge_index=[2, 162], x=[42, 5], y=[1])

In [33]:
class GCNConv_local(gnn.MessagePassing):
    # Implementation from troch.geometric
    
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='add')  # "Add" aggregation (Step 5).

        '''
        TODO: implement the GCN layer with a bias according the
        equations given.
        Arguments:
        in_channels: dimension of input
        out_channels: dimension of output
        '''
        ### Start your code here
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.bias = nn.Parameter(torch.Tensor(out_channels))
        ### End your code

        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        self.bias.data.zero_()

    def forward(self, x, edge_index):
        '''
        arguments:
        x: the feature matrix of the graph.
        x has shape [N, in_channels], where N is the number of nodes
        edge_index: the adjacency list of the graph.
        edge_index has shape [2, E]
        '''

        ### TODO: implement the forward pass
        # Step 1: Add self-loops to the adjacency matrix.
        # hint, we imported an add_self_loops function.

        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Apply the linear transofmration.
        x = self.lin(x)
        
        
        # Step 3: Compute the normalized Laplacian.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Propagate messages
        out = self.propagate(edge_index, x=x, norm=norm)

        # Step 6: Apply the bias
        out += self.bias

        ###END TODO

        return out

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]
        
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

In [34]:
class Model(nn.Module):
    def __init__(self, num_features, num_classes):
        super(Model, self).__init__()

        self.conv1 = GCNConv(num_features, 32)
        self.conv2 = GCNConv(32, 32)
        self.conv3 = GCNConv(32, 32)
        self.conv4 = GCNConv(32, 1)
        self.conv5 = Conv1d(1, 16, 97, 97)
        self.conv6 = Conv1d(16, 32, 5, 1)
        self.pool = MaxPool1d(2, 2)
        self.classifier_1 = Linear(352, 128)
        self.drop_out = Dropout(0.5)
        self.classifier_2 = Linear(128, num_classes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        edge_index, _ = remove_self_loops(edge_index)

        x_1 = torch.tanh(self.conv1(x, edge_index))
        x_2 = torch.tanh(self.conv2(x_1, edge_index))
        x_3 = torch.tanh(self.conv3(x_2, edge_index))
        x_4 = torch.tanh(self.conv4(x_3, edge_index))
        x = torch.cat([x_1, x_2, x_3, x_4], dim=-1)
        x = global_sort_pool(x, batch, k=30)
        x = x.view(x.size(0), 1, x.size(-1))
        x = self.relu(self.conv5(x))
        x = self.pool(x)
        x = self.relu(self.conv6(x))
        x = x.view(x.size(0), -1)
        out = self.relu(self.classifier_1(x))
        out = self.drop_out(out)
        classes = F.log_softmax(self.classifier_2(out), dim=-1)

        return classes